In [1]:
import os
import requests
import json
import openai
import pandas as pd
import time
openai.api_key = "“
openai.api_base = ""
openai.api_type = ''
openai.api_version = ''

In [2]:
data = pd.read_csv('data/mc.csv',index_col=0)
data.head()

,id,goal,sol1,sol2,label,slot_question,slot_answer
0,fe68f9ec-09fd-436e-bcaf-07863711ec2b,dresser,replace drawer with bobby pin,"finish, woodgrain with bobby pin",1,"dresser\nsol1:\nsol2:finish, woodgrain with b...",replace drawer with bobby pin
1,d73182e6-6916-48a0-b31f-2137e350776f,To fight Ivan Drago in Rocky for sega master s...,Drago isn't in this game because it was releas...,You have to defeat Apollo Creed and Clubber La...,1,To fight Ivan Drago in Rocky for sega master s...,Drago isn't in this game because it was releas...
2,f8232939-914b-4a5a-af72-dc37cd042263,How do you attach toilet paper to a glass jar?,Press a piece of double-sided tape to the glas...,Spread mayonnaise all over the jar with your p...,0,How do you attach toilet paper to a glass jar?...,Spread mayonnaise all over the jar with your p...
3,8748b18c-01df-4544-96ca-1defe1b182b7,how do you put eyelashes on?,glue them on with mascara.,put eyelash glue on the fake eyelashes and the...,1,how do you put eyelashes on?\nsol1:\nsol2:put ...,glue them on with mascara.
4,f1386ea3-ee1e-4395-a760-747e37b11c30,Find spices easily in the kitchen.,Arrange spices from hot to mild in the kitchen...,Arrange your spices alphabetically to make fin...,1,Find spices easily in the kitchen.\nsol1:\nsol...,Arrange spices from hot to mild in the kitchen...


In [35]:
ans_list = {}

In [54]:
data = data.drop(242)
data = data.reset_index(drop=True)

In [57]:
for index, row in data[347:].iterrows():
    init_prompt = "Help me complete the missing solution.\nHere are some examples:\nWhen boiling butter, when it's ready, you can\nsol1: Pour it onto a plate \nsol2: \nAnswer:Pour it into a jar\n"
    prompt = init_prompt + f"{row['slot_question']}\nAnswer:"
    # print(prompt)
    try:
        response = openai.ChatCompletion.create(
            engine="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"{prompt}"},
    
            ]
        )
    except openai.error.RateLimitError:
            time.sleep(10)
            response = openai.ChatCompletion.create(
            engine="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"{prompt}"},
            ]
        )
    print(index,response['choices'][0]['message']['content'])
    ans_list[row['slot_question']] = response['choices'][0]['message']['content']

347 Set up a water source like a small pond or birdbath.
348 Purchase a ripe watermelon from the store
349 Grab the ball and pull it back, then let it go to start the swinging motion.
350 Store them in an airtight container in the refrigerator.
351 Flush waste
352 Use a very sharp knife to ensure clean cuts without tearing the meat.
353 Use a long-handled duster, reaching up to swipe each blade from base to tip.
354 Cut the grapefruit in half, use a grapefruit spoon to scoop out each section and eat it.
355 You should read the instruction manual carefully.
356 Olive oil
357 Snap any additional buckles or closures to secure the contents inside.
358 Install a lock that can be unlocked from both sides
359 Marinate it beforehand for extra flavor
360 Melt the butter and brush it over the toast.
361 Install a fence around the beach
362 Soak it in a solution of bleach and water
363 Nail it to a wall
364 hydrate your body
365 Pressing the power button on the desktop tower.
366 Prepare thorough

In [58]:
data['pred_answer'] = ans_list.values()
data.to_csv("result/mc_gpt_4.csv")

In [59]:
data.to_csv("result/mc_gpt_4.csv")